# Analyse de la fraude triangulaire dans les données de trajets de covoiturage

[Introduction](#introduction)
  - Présentation du contexte et de l'objectif de l'analyse
  - Explication des données de trajets de covoiturage utilisées
  
[Chargement des données](#data)
 - Importation des bibliothèques nécessaires
 - Chargement des données de l'AOM et de la période spécifiées

[Détection des paires de changement de rôle](#pairs)
  - Création d'une fonction pour rechercher les paires de participants qui changent de rôle dans les trajets
  - Application de la fonction aux données de trajets pour identifier les paires suspectes

[Analyse des groupes frauduleux avec les graphes de connexion](#groups)
  - Création d'un graphe de connexion à partir des groupes suspects
  - Identification des groupes de participants frauduleux à l'aide des composantes connexes du graphe
  - Calcul des mesures de centralité pour évaluer l'importance des participants dans les groupes

[Rapport sur les groupes frauduleux détectés](#report)
 - Présentation des groupes frauduleux identifiés avec leurs caractéristiques (nombre de participants, durée des trajets, etc.)
 - Statistiques descriptives sur les groupes (nombre moyen de participants, durée moyenne des trajets, etc.)

[Conclusion](#conclusions)
 - Récapitulatif des principales conclusions de l'analyse
 - Discussion sur les implications et les recommandations éventuelles

## [Introduction](#introduction)
### Présentation du contexte et de l'objectif de l'analyse

Dans le cadre de cette analyse, nous nous intéressons aux données de trajets de covoiturage. Le covoiturage est devenu une alternative populaire pour les déplacements, offrant un moyen économique et écologique de se déplacer d'un endroit à un autre. Cependant, comme toute activité impliquant des interactions entre individus, il peut également être sujet à des comportements frauduleux.

L'objectif de cette analyse est de détecter les groupes de participants impliqués dans des activités frauduleuses dans les trajets de covoiturage. Nous cherchons à identifier les participants qui changent de rôle de manière suspecte, c'est-à-dire ceux qui passent de conducteur à passager ou vice versa de manière répétée et inhabituelle. En détectant ces paires de participants, nous pouvons explorer les connexions entre eux pour identifier les groupes frauduleux et en comprendre les schémas de comportement.

### Explication des données de trajets de covoiturage utilisées

Les données de trajets de covoiturage utilisées dans cette analyse comprennent des informations sur les trajets effectués dans une région spécifique pendant une période donnée. Chaque trajet est caractérisé par des détails tels que l'identifiant du trajet, la durée, la date et l'heure, l'identifiant de l'opérateur, l'identifiant du trajet ainsi que les numéros de téléphone tronqués des participants.

Ces données nous permettent de reconstituer les trajets effectués par les participants et d'analyser les interactions entre eux. En nous concentrant sur les changements de rôle suspects, nous pouvons détecter les groupes de participants qui pourraient être impliqués dans des activités frauduleuses.


## [Chargement des données](#data)
### Importation des bibliothèques nécessaires

In [ ]:
import warnings
import os
import pandas as pd
import psycopg2
import numpy as np
import ast
import networkx as nx
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()

### Chargement des données de l'AOM et de la période spécifiées

In [ ]:
start_date = "2023-02-27 23:59:59"
end_date = "2023-04-30 00:00:01"
aom_insee = "217500016"

DATABASE = os.environ['DATABASE']
USER = os.environ['USER_DB']
PASSWORD = os.environ['PASSWORD']
PORT = os.environ['PORT']
con = psycopg2.connect(
    host=os.environ['HOST'],
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    port=PORT,
    sslmode="require"
)

In [ ]:
query = f"""SELECT cc._id, cc.is_driver, ci.phone_trunc, cc.datetime, cc.duration, cc.operator_id, cc.start_position, cc.operator_journey_id,cc.start_geo_code,cc.end_geo_code,
cc.end_position, gmap_url(cc.start_position, cc.end_position), CASE WHEN cc.is_driver THEN pa.rpc_driver_uuid ELSE pa.rpc_passenger_uuid END as rpc_uuid, pi.result
FROM CARPOOL.CARPOOLS cc
   join carpool.identities ci on cc.identity_id = ci._id
   join geo.perimeters gps on cc.start_geo_code = gps.arr and gps.year = 2022
   join geo.perimeters gpe on cc.end_geo_code = gpe.arr and gpe.year = 2022
   LEFT join phones.all pa on pa.operator_journey_id = cc.operator_journey_id
   LEFT JOIN policy.incentives pi on pi.carpool_id = cc._id and pi.policy_id = 459
WHERE CC.DATETIME >= '{start_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    AND cc.operator_id IN (3,4,9)
	AND CC.DATETIME < '{end_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    {f"and (gps.aom = '{aom_insee}' or gpe.aom = '{aom_insee}') and gps.year = 2022 and gpe.year = 2022" if aom_insee else ""}
"""
df_carpool = pd.read_sql(query, con)
df_carpool.head(10)   

In [ ]:
df_carpool.columns

In [ ]:
df_carpool.shape

## [Détection des paires de changement de rôle](pairs)
### Création d'une fonction pour rechercher les paires de participants qui changent de rôle dans les trajets

#### Méthdodologie :
 - Ajouter colonne conducteur ou passager en fonction de 'is_driver'.
 - Création variable unique phone_trunc avec rôle (conducteur ou passager).
 - Enumérer la liste des uniques phone_trunc avec rôle pour chaque operator_journey_id.
 - Aggréger par pair de phone_trunc.
 - Filtrer les pairs de phone_trunc qui ont une liste de rôle unique de taille plus grande que 2 = changement de rôle.

In [ ]:
def find_potential_triangular_pairs(df):   
    get_role = lambda x: 'driver' if x else 'passenger'
    df['role'] = df['is_driver'].apply(get_role)
    df['temp'] = df['phone_trunc'] +'_'+ df['role']
    _temp = df.groupby('operator_journey_id').agg(lambda x: sorted(list(x))).reset_index()
    _temp['phone_trunc'] = _temp['phone_trunc'].astype(str)
    grouped = _temp.groupby('phone_trunc')[['temp','operator_journey_id']].agg(lambda x: list(x)).reset_index()
    _temp_1 = pd.DataFrame(grouped['temp'].explode().explode())
    _temp_1.reset_index(inplace=True)
    _temp_1.groupby('index').agg({'temp' : 'nunique'}).reset_index().temp
    grouped['roles_list'] = _temp_1.groupby('index').agg({'temp' : 'nunique'}).reset_index().temp
    potential_triangular = lambda x: True if x > 2 else False
    grouped['potential_triangular'] = grouped.roles_list.apply(potential_triangular)

    return grouped

### Application de la fonction aux données de trajets pour identifier les paires suspectes

In [ ]:
triangular_df = find_potential_triangular_pairs(df_carpool)
triangular_df.head()

In [ ]:
# % de trajets potentiellement triangulaire
len(triangular_df[triangular_df.potential_triangular == True])/len(triangular_df)*100

## [Analyse des groupes frauduleux avec les graphes de connexion](#groups)

### Création d'un graphe de connexion à partir des groupes suspects

In [ ]:
# filtrer df_carpool sur les phone_trunc potentiel à la fraude triangulaire
phone_numbers = triangular_df[triangular_df.potential_triangular ==  True]['phone_trunc'].to_list()
phone_truncs = []
for item in phone_numbers:
    numbers = ast.literal_eval(item)
    phone_truncs.extend(numbers)
    # Filter the original dataset based on phone truncs
filtered_df = df_carpool[df_carpool['phone_trunc'].isin(phone_truncs)]
filtered_df

In [ ]:
# petite vérif
set(filtered_df.phone_trunc.unique()) == set(np.unique(phone_truncs))

In [ ]:
filtered_df.columns

In [ ]:
filtered_df_grouped = filtered_df.groupby(['operator_journey_id','duration','datetime','operator_id']).agg({'phone_trunc' : list })
filtered_df_grouped.reset_index(inplace=True)
filtered_df_grouped

In [ ]:
# graph for only the potential triangular for pairs
G = nx.Graph()
# Add edges between connected phone trunc
for _, row in filtered_df_grouped.iterrows():
    phone_list = row['phone_trunc']#.replace("'", "").replace("[", "").replace("]", "").split(", ")
    for i in range(len(phone_list) - 1):
        for j in range(i + 1, len(phone_list)):
            G.add_edge(phone_list[i], phone_list[j])

# Find connected components in the graph
connected_components = [component for component in nx.connected_components(G) if len(component) == 2]

# Visualize the groups
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

# Draw nodes and edges
for component in connected_components:
    nx.draw_networkx_nodes(G, pos, nodelist=list(component), node_size=200, alpha=0.8)
    nx.draw_networkx_edges(G, pos, edgelist=list(G.subgraph(component).edges()), width=1.5, alpha=0.5)

# Customize plot appearance
plt.axis('off')
plt.title('Fraudulent Pairs')
plt.show()


In [ ]:
# graph for only the potential triangular for pairs
G = nx.Graph()
# Add edges between connected phone trunc
for _, row in filtered_df_grouped.iterrows():
    phone_list = row['phone_trunc']#.replace("'", "").replace("[", "").replace("]", "").split(", ")
    for i in range(len(phone_list) - 1):
        for j in range(i + 1, len(phone_list)):
            G.add_edge(phone_list[i], phone_list[j])

# Find connected components in the graph
connected_components = [component for component in nx.connected_components(G) if len(component) > 2]

# Visualize the groups
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

# Draw nodes and edges
for component in connected_components:
    nx.draw_networkx_nodes(G, pos, nodelist=list(component), node_size=200, alpha=0.8)
    nx.draw_networkx_edges(G, pos, edgelist=list(G.subgraph(component).edges()), width=1.5, alpha=0.5)

# Customize plot appearance
plt.axis('off')
plt.title('Fraudulent Groups (more than 2)')
plt.show()


In [ ]:
# graph for only the potential triangular for pairs
G = nx.Graph()
# Add edges between connected phone trunc
for _, row in filtered_df_grouped.iterrows():
    phone_list = row['phone_trunc']#.replace("'", "").replace("[", "").replace("]", "").split(", ")
    for i in range(len(phone_list) - 1):
        for j in range(i + 1, len(phone_list)):
            G.add_edge(phone_list[i], phone_list[j])

# Find connected components in the graph
connected_components = [component for component in nx.connected_components(G) if len(component) > 4]

# Visualize the groups
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

# Draw nodes and edges
for component in connected_components:
    nx.draw_networkx_nodes(G, pos, nodelist=list(component), node_size=200, alpha=0.8)
    nx.draw_networkx_edges(G, pos, edgelist=list(G.subgraph(component).edges()), width=1.5, alpha=0.5)

# Customize plot appearance
plt.axis('off')
plt.title('Fraudulent Groups (more than 4)')
plt.show()


In [ ]:
G = nx.Graph()

# Add edges between connected phone trunc
for _, row in filtered_df_grouped.iterrows():
    phone_list = row['phone_trunc']
    for i in range(len(phone_list) - 1):
        for j in range(i + 1, len(phone_list)):
            if G.has_edge(phone_list[i], phone_list[j]):
                G[phone_list[i]][phone_list[j]]['interactions'] += 1
            else:
                G.add_edge(phone_list[i], phone_list[j], interactions=1)

# Find connected components in the graph
connected_components = [component for component in nx.connected_components(G) if len(component) > 20]

# Visualize the groups
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42, k=0.1)

# Calculate line widths based on interactions
edge_widths = [0.5 * G[u][v]['interactions'] for u, v in G.edges()]

# Draw nodes and edges
for component in connected_components:
    nx.draw_networkx_nodes(G, pos, nodelist=list(component), node_size=100, alpha=0.8)
    nx.draw_networkx_edges(G, pos, edgelist=list(G.subgraph(component).edges()), width=edge_widths, alpha=0.5)

# Add labels to nodes
labels = {node: node for node in G.nodes() if node in connected_components}
nx.draw_networkx_labels(G, pos, labels, font_size=10, font_color='white')

# Customize plot appearance
plt.axis('off')
plt.title('Fraudulent Groups (more than twenty)')
plt.show()

## [Rapport sur les groupes frauduleux détectés](#report)

### Présentation des groupes frauduleux identifiés avec leurs caractéristiques (nombre de participants, durée des trajets, etc.)

In [ ]:
G = nx.Graph()

# Add edges between connected phone trunc
for _, row in filtered_df_grouped.iterrows():
    phone_list = row['phone_trunc']
    for i in range(len(phone_list) - 1):
        for j in range(i + 1, len(phone_list)):
            if G.has_edge(phone_list[i], phone_list[j]):
                G[phone_list[i]][phone_list[j]]['interactions'] += 1
            else:
                G.add_edge(phone_list[i], phone_list[j], interactions=1)

# Find connected components in the graph
#connected_components = [component for component in nx.connected_components(G) if len(component) > 2]
connected_components = nx.connected_components(G)

# Create DataFrame with groups
group_data = []
group_degree_centrality = []
group_betweenness_centrality = []

for idx, component in enumerate(connected_components):
    group_graph = G.subgraph(component)
    degree_centrality = nx.degree_centrality(group_graph)
    betweenness_centrality = nx.betweenness_centrality(group_graph)

    group_phones = list(component)
    
    group_journeys = df_carpool[df_carpool['phone_trunc'].isin(group_phones)]
    group_duration = group_journeys['duration'].mean()//60
    group_operator_id = group_journeys['operator_journey_id']
    group_incentives = group_journeys['result']
    group_start_date = group_journeys['datetime'].min().date()
    group_end_date = group_journeys['datetime'].max().date()
    group_journeys['date'] = group_journeys['datetime'].dt.date
    grouped = group_journeys.groupby('phone_trunc').size().reset_index(name='count')
    
    group_data.append({
        'Group': idx+1,
        'Phone Numbers': group_phones,
        'Phone Trunc Count': len(group_phones),
        'Journeys Count': len(group_journeys.operator_journey_id.unique()),
        'Operator Id' : group_journeys.operator_id.unique(),
        'Number of Operator Id' : len(group_journeys.operator_id.unique()),
        'Mean duration (min)': group_duration,
        'Start Date': group_start_date,
        'End Date': group_end_date,
        'Daily mean trips' : group_journeys.drop_duplicates('operator_journey_id')['datetime'].dt.date.value_counts().sort_index().mean(),
        'Participant(s) central' : degree_centrality,
        'Participant(s) intermédiare' : betweenness_centrality,
        'operator_journey_id' : group_operator_id,
        'incentives' : group_incentives,
        'id_max_occurence': grouped.loc[grouped['count'].idxmax(),'phone_trunc'],
        'mean_per_day_id_max_occurence' : grouped.loc[grouped['count'].idxmax(),'count']/len(group_journeys.groupby('date'))
      
    })

groups_df = pd.DataFrame(group_data)

# Print the DataFrame
groups_df


In [ ]:
# groups_df.to_csv('./fraude_triangulaire_agg.csv',sep=';',index=False)

In [ ]:
fraud_phone_trunc_3 = groups_df[groups_df['mean_per_day_id_max_occurence']>= 3]['Phone Numbers'].explode()
raude_triangulaire_3_journey_id_list = groups_df[groups_df['mean_per_day_id_max_occurence']>= 3]['operator_journey_id'].explode()

In [ ]:
groups_df[groups_df['mean_per_day_id_max_occurence']>= 3]['operator_journey_id'].explode()

In [ ]:
fraude_triangulaire_3_journey_id_df= df_carpool[df_carpool.operator_journey_id.isin(fraude_triangulaire_3_journey_id_list)]
fraude_triangulaire_3_journey_id_df = fraude_triangulaire_3_journey_id_df.drop_duplicates('operator_journey_id').copy()
#fraude_triangulaire_3_journey_id_df.to_csv('fraude_triangulaire_3_journey_id.csv',sep=';',index=False)

### Statistiques descriptives sur les groupes 

In [ ]:
groups_df['Operator Id'].value_counts()

In [ ]:
groups_df.sort_values('Daily mean trips',ascending=True)

In [ ]:
groups_df['mean_per_person'] = groups_df['Daily mean trips']/groups_df['Phone Trunc Count']

In [ ]:
groups_df.sort_values('mean_per_person',ascending=False)

In [ ]:
test = groups_df[groups_df.mean_per_person >= 3]

In [ ]:
test.sort_values('Phone Trunc Count',ascending=False)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Get the group with the highest Journeys Count
group_max_journeys = groups_df[groups_df['Journeys Count'] == groups_df['Journeys Count'].max()]

# Get the relevant data for radar plot
labels = ['Phone Trunc Count', 'Number of Operator Id', 'Daily Mean Trips']
values = group_max_journeys[['Phone Trunc Count','Number of Operator Id', 'Daily mean trips']].values.flatten()

# Perform Min-Max scaling on the values
scaler = MinMaxScaler(feature_range=(0, 1))
values_scaled = scaler.fit_transform(values.reshape(-1, 1)).flatten()



# Extend the scaled values array to match the length of labels
values_scaled = np.append(values_scaled, values_scaled[0])

# Calculate the angle for each axis
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]

# Create the radar plot
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(polar=True))
ax.fill(angles, values_scaled, color='skyblue', alpha=0.5)
ax.plot(angles, values_scaled, color='blue', linewidth=1.5)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels)
ax.yaxis.grid(True)

# Add a title
plt.title('Radar Plot - Group with Highest Journeys Count', fontsize=12)

# Display the plot
plt.show()